###### Задание 1  
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
#Решение по Заданию 1.

import pandas as pd
MOVIES_PATH  = "source/ml-latest-small/movies.csv"
RATINGS_PATH = "source/ml-latest-small/ratings.csv"

#Функция для классификации диапозона значений
def rating_gr(rating):
    if rating <= 2:
        ret = "низкий рейтинг"
    elif  2 < rating <=  4:
        ret = "средний рейтинг"
    else:
        ret = "высокий рейтинг"
    return ret

#Получение фрейма с названием фильма по его идентификатору
df_join_movies_ratings = pd.read_csv(MOVIES_PATH).merge(pd.read_csv(RATINGS_PATH), how = 'inner', on = 'movieId')

df_join_movies_ratings['class'] = df_join_movies_ratings['rating'].apply(rating_gr)

print('Результат:\n\n', df_join_movies_ratings.groupby(by = ['class', 'rating']).count()['userId'].index)
df_join_movies_ratings.head(3)

Результат:

 MultiIndex([('высокий рейтинг', 4.5),
            ('высокий рейтинг', 5.0),
            ( 'низкий рейтинг', 0.5),
            ( 'низкий рейтинг', 1.0),
            ( 'низкий рейтинг', 1.5),
            ( 'низкий рейтинг', 2.0),
            ('средний рейтинг', 2.5),
            ('средний рейтинг', 3.0),
            ('средний рейтинг', 3.5),
            ('средний рейтинг', 4.0)],
           names=['class', 'rating'])


,movieId,title,genres,userId,rating,timestamp,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703,средний рейтинг
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179,средний рейтинг
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058,высокий рейтинг


###### Задание 2  
Используем файл keywords.csv.

- Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток: словарь geo_data
Результат классификации запишите в отдельный столбец region.

In [26]:
page_url = 'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
df = pd.read_html(page_url)[0]
df.columns = ['num', 'logo', 'name', 'region', 'federal', 'piople', 'base', 'status', 'name_old']

#формируем новый словарь geo_data на основе данных о городах и Википедии
geo_data = {}
list(df['federal'].unique())

for item in list(df['federal'].unique()):
    geo_data[item] = list(df[(df['federal'] == item)]['name'].str.lower())

geo_data.keys()

dict_keys(['Сибирский', 'Приволжский', 'Южный', 'Северо-Кавказский', 'Уральский', 'Дальневосточный', 'Центральный', 'Северо-Западный'])

In [27]:
%%time
#Решение по Заданию 2.
KEYWORDS_PATH = "source/ml-latest-small/keywords.csv"
# Функция для преобразования словарей типа geo_data в словарь вида {'Город': 'Регион'}
def transform_geo_data(dict_):
    ret = {}
    for key, values_ in dict_.items():
        for value_ in set(values_):
            ret[value_] = key
    return ret
# Функция для находждения городов в строке запроса, используя пересечение множеств,
def find_intersection(words, region1_cityes, city1_region1):
    ret = 'undefined'
    words_set = set(words.split(' '))
    for key, values_ in region1_cityes.items():
        intersec_tmp = words_set & set(values_)
        if intersec_tmp:
            if list(intersec_tmp)[0] in city1_region1.keys():
                ret = city1_region1[list(intersec_tmp)[0]]
    return(ret)
GEO_DATA_TRANSFORM =  transform_geo_data(geo_data)
df_keywords = pd.read_csv(KEYWORDS_PATH)
df_keywords['region'] = ( df_keywords['keyword'].apply(find_intersection,
                                                       region1_cityes = geo_data,
                                                       city1_region1 = GEO_DATA_TRANSFORM) )
print('Проверка результата:')
df_keywords[(df_keywords['region'].isin(df['federal']) )]


Проверка результата:
Wall time: 7.41 s


,keyword,shows,region
127,авито москва,979292,Центральный
362,авито самара,440746,Приволжский
390,авито волгоград,408970,Южный
396,авито пермь,406796,Приволжский
434,авито саратов,377723,Приволжский
...,...,...,...
99897,окей уфа каталог товаров цены уфа акции,3648,Приволжский
99922,авто ру ярославль,3712,Центральный
99925,дром северобайкальск,3734,Сибирский
99960,погода в котельниково,3559,Южный


In [28]:
%%time
#Решение по Заданию 2. Вариант, предложенный преподавателем

def geo_classificator(keyword):
    # проходим по ключам (название региона) и значениям (список городов региона) словаря geo_data
    for region, city_list in geo_data.items():
        # проходим по каждому городу региона
        for city in city_list:
            # если название города есть в поисковом запросе, то возвращаем название региона region
            if city in keyword:
                return region
    
    # если ничего не нашлось, то возвращаем undefined
    return 'undefined'

df_keywords = pd.read_csv(KEYWORDS_PATH)

df_keywords['region'] = df_keywords['keyword'].apply( geo_classificator )
print('Проверка результата:')
df_keywords[(df_keywords['region'].isin(df['federal']) )]

Проверка результата:
Wall time: 14.8 s


,keyword,shows,region
1,одноклассники,63810309,Северо-Западный
5,одноклассники моя страница,19976138,Северо-Западный
25,одноклассники социальная сеть,4018230,Северо-Западный
58,табор,2014357,Приволжский
79,маша и медведь,1555439,Уральский
...,...,...,...
99925,дром северобайкальск,3734,Сибирский
99937,варвара визбор ты мое дыхание,5537,Приволжский
99960,погода в котельниково,3559,Южный
99973,медиа маркт оренбург официальный сайт,3689,Приволжский


###### Задание 3 (бонусное)  
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из [базы](https://grouplens.org/datasets/movielens)). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:  
1. В переменную years запишите список из всех годов с 1950 по 2010.
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
   - для каждой строки пройдите по всем годам списка years
       - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
       - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’  
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [3]:
#Решение по Заданию 3.
import datetime as dt
import re
years = list(range(1950, 2011))

def production_year(par):
    #регулярное выражение для поиска 4-х цифр в скобках (dddd) возвращает 4-х зн. число
    tmp = re.findall(r'\((\d{4})\)', par)
    ret = 1900
    if tmp:
        ret = int(tmp[0])
        if ret not in years:
            ret = 1900
    return ret

df_movies  = pd.read_csv(MOVIES_PATH)
df_movies['year'] = df_movies['title'].apply(production_year)

df_ratings = df_movies.merge(pd.read_csv(RATINGS_PATH), how = 'inner', on = 'movieId')

df_ratings.groupby(by = 'year')['rating'].mean().reset_index().sort_values(by = 'rating', ascending = False)

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
...,...,...
56,2005,3.448434
54,2003,3.444777
47,1996,3.426600
48,1997,3.415764
